In [1]:
# 📘 Personal Finance Dashboard for MoneyWise Solutions
print('Hello from YBI Foundation 🚀')

Hello from YBI Foundation 🚀


In [1]:
# STEP 1: IMPORT REQUIRED LIBRARIES
# ============================================================================
import pandas as pd        # For data analysis and CSV operations
import uuid               # For generating unique transaction IDs
from datetime import datetime  # For timestamp creation
import os

In [2]:
# STEP 2: INITIALIZE DATA STRUCTURES
# ============================================================================

# Main data storage - list of tuples to store all transactions
transactions = []

# Set to store unique transaction IDs (prevents duplicates)
transaction_ids = set()

# Define valid transaction categories
INCOME_CATEGORIES = [
    'Salary',
    'Freelance',
    'Investment',
    'Business',
    'Other Income'
]

EXPENSE_CATEGORIES = [
    'Food',
    'Transportation',
    'Housing',
    'Utilities',
    'Entertainment',
    'Healthcare',
    'Shopping',
    'Education',
    'Other Expenses'
]


In [3]:
# STEP 3: VALIDATION FUNCTIONS
# ============================================================================

def validate_amount(amount):
    """
    Validate transaction amount
    Args:
        amount: The amount to validate
    Returns:
        float: Validated positive amount
    Raises:
        ValueError: If amount is invalid or non-positive
    """
    try:
        amount = float(amount)
        if amount <= 0:
            raise ValueError("Amount must be positive")
        return amount
    except ValueError as e:
        raise ValueError(f"Invalid amount: {e}")

def generate_transaction_id():
    """
    Generate a unique 8-character transaction ID
    Returns:
        str: Unique transaction ID
    """
    while True:
        # Generate 8-character ID from UUID
        transaction_id = str(uuid.uuid4())[:8]

        # Check if ID is unique
        if transaction_id not in transaction_ids:
            transaction_ids.add(transaction_id)
            return transaction_id


In [4]:
# STEP 4: TRANSACTION MANAGEMENT FUNCTIONS
# ============================================================================

def add_transaction(transaction_type, amount, category, description=""):
    """
    Add a new transaction to the system
    Args:
        transaction_type (str): 'Income' or 'Expense'
        amount (float): Transaction amount
        category (str): Transaction category
        description (str): Optional description
    Returns:
        str: Transaction ID if successful, None if failed
    """
    try:
        # Validate transaction type
        if transaction_type not in ['Income', 'Expense']:
            raise ValueError("Transaction type must be 'Income' or 'Expense'")

        # Validate amount
        validated_amount = validate_amount(amount)

        # Validate category based on transaction type
        valid_categories = INCOME_CATEGORIES if transaction_type == 'Income' else EXPENSE_CATEGORIES
        if category not in valid_categories:
            raise ValueError(f"Invalid category for {transaction_type}. Choose from: {valid_categories}")

        # Generate unique ID and timestamp
        transaction_id = generate_transaction_id()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Create transaction tuple
        transaction = (
            transaction_id,      # Unique identifier
            timestamp,           # When transaction was recorded
            transaction_type,    # Income or Expense
            validated_amount,    # Amount (validated)
            category,           # Category
            description         # Optional description
        )

        # Add to transactions list
        transactions.append(transaction)
        print(f"✅ Transaction added successfully! ID: {transaction_id}")
        return transaction_id

    except ValueError as e:
        print(f"❌ Error: {e}")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None

def view_transactions(limit=10):
    """
    Display recent transactions
    Args:
        limit (int): Number of recent transactions to show
    """
    if not transactions:
        print("📭 No transactions found.")
        return

    print(f"\n📊 Recent Transactions (Last {min(limit, len(transactions))})")
    print("=" * 85)

    # Display transactions (most recent first)
    for transaction in transactions[-limit:]:
        transaction_id, timestamp, t_type, amount, category, description = transaction

        # Format output
        type_symbol = "💰" if t_type == "Income" else "💸"
        print(f"{type_symbol} ID: {transaction_id} | {timestamp}")
        print(f"   Type: {t_type} | Amount: ${amount:,.2f} | Category: {category}")

        if description:
            print(f"   Description: {description}")
        print("-" * 85)


In [5]:
# STEP 5: DATA ANALYSIS FUNCTIONS (USING PANDAS)
# ============================================================================

def get_financial_summary():
    """
    Generate comprehensive financial summary using Pandas
    Returns:
        DataFrame: Transaction data for further analysis
    """
    if not transactions:
        print("📭 No transactions to summarize.")
        return None

    # Convert transactions list to Pandas DataFrame
    df = pd.DataFrame(transactions, columns=[
        'Transaction_ID',
        'Timestamp',
        'Type',
        'Amount',
        'Category',
        'Description'
    ])

    # Convert timestamp to datetime for better analysis
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])

    # Calculate financial totals
    total_income = df[df['Type'] == 'Income']['Amount'].sum()
    total_expenses = df[df['Type'] == 'Expense']['Amount'].sum()
    net_balance = total_income - total_expenses

    # Generate category-wise summary
    category_summary = df.groupby(['Type', 'Category'])['Amount'].agg(['sum', 'count']).reset_index()
    category_summary.columns = ['Type', 'Category', 'Total_Amount', 'Transaction_Count']

    # Display summary
    print("\n💰 FINANCIAL SUMMARY")
    print("=" * 60)
    print(f"💵 Total Income:      ${total_income:,.2f}")
    print(f"💸 Total Expenses:    ${total_expenses:,.2f}")
    print(f"📊 Net Balance:       ${net_balance:,.2f}")
    print(f"📝 Total Transactions: {len(transactions)}")

    # Balance status
    if net_balance > 0:
        print("✅ Status: Positive cash flow")
    elif net_balance < 0:
        print("⚠️  Status: Negative cash flow")
    else:
        print("➖ Status: Break-even")

    print("\n📈 SUMMARY BY CATEGORY:")
    print("=" * 60)
    print(category_summary.to_string(index=False))

    return df

def get_monthly_summary():
    """
    Generate monthly breakdown of transactions
    """
    if not transactions:
        print("📭 No transactions for monthly summary.")
        return

    # Convert to DataFrame
    df = pd.DataFrame(transactions, columns=[
        'Transaction_ID', 'Timestamp', 'Type', 'Amount', 'Category', 'Description'
    ])

    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df['Month'] = df['Timestamp'].dt.to_period('M')

    # Monthly summary
    monthly_summary = df.groupby(['Month', 'Type'])['Amount'].sum().unstack(fill_value=0)

    if not monthly_summary.empty:
        monthly_summary['Net'] = monthly_summary.get('Income', 0) - monthly_summary.get('Expense', 0)

        print("\n📅 MONTHLY SUMMARY:")
        print("=" * 50)
        print(monthly_summary.to_string())

In [6]:
# STEP 6: DATA EXPORT FUNCTIONS
# ============================================================================

def export_to_csv(filename="moneywise_transactions.csv"):
    """
    Export all transactions to CSV file using Pandas
    Args:
        filename (str): Name of the CSV file to create
    """
    if not transactions:
        print("📭 No transactions to export.")
        return

    try:
        # Convert transactions to DataFrame
        df = pd.DataFrame(transactions, columns=[
            'Transaction_ID',
            'Timestamp',
            'Type',
            'Amount',
            'Category',
            'Description'
        ])

        # Export to CSV
        df.to_csv(filename, index=False)

        print(f"✅ Transactions exported successfully!")
        print(f"📁 File: {filename}")
        print(f"📍 Location: {os.path.abspath(filename)}")
        print(f"📊 Records exported: {len(transactions)}")

    except Exception as e:
        print(f"❌ Error exporting to CSV: {e}")

In [7]:
# STEP 7: UTILITY FUNCTIONS
# ============================================================================

def show_categories():
    """Display available transaction categories"""
    print("\n📋 AVAILABLE CATEGORIES")
    print("=" * 40)

    print("💰 INCOME CATEGORIES:")
    for i, category in enumerate(INCOME_CATEGORIES, 1):
        print(f"  {i}. {category}")

    print("\n💸 EXPENSE CATEGORIES:")
    for i, category in enumerate(EXPENSE_CATEGORIES, 1):
        print(f"  {i}. {category}")

def get_transaction_stats():
    """Display transaction statistics"""
    if not transactions:
        print("📭 No transactions for statistics.")
        return

    income_count = sum(1 for t in transactions if t[2] == 'Income')
    expense_count = sum(1 for t in transactions if t[2] == 'Expense')

    print(f"\n📈 TRANSACTION STATISTICS")
    print("=" * 40)
    print(f"📝 Total Transactions: {len(transactions)}")
    print(f"💰 Income Transactions: {income_count}")
    print(f"💸 Expense Transactions: {expense_count}")
    print(f"🆔 Unique IDs Generated: {len(transaction_ids)}")

# STEP 8: INTERACTIVE MENU SYSTEM
# ============================================================================

def display_menu():
    """Display the main menu options"""
    print("\n🏦 MONEYWISE SOLUTIONS - FINANCE DASHBOARD")
    print("=" * 55)
    print("1️⃣  Add Income Transaction")
    print("2️⃣  Add Expense Transaction")
    print("3️⃣  View Recent Transactions")
    print("4️⃣  Generate Financial Summary")
    print("5️⃣  Generate Monthly Summary")
    print("6️⃣  Export to CSV")
    print("7️⃣  Show Categories")
    print("8️⃣  Transaction Statistics")
    print("9️⃣  Exit")
    print("=" * 55)

def main():
    """Main program loop"""
    print("🏦 Welcome to MoneyWise Solutions Finance Dashboard!")

    while True:
        display_menu()

        try:
            choice = input("\n🔹 Enter your choice (1-9): ").strip()

            if choice == '1':
                # Add Income
                print("\n💰 Adding Income Transaction")
                show_categories()
                amount = input("💵 Enter amount: $")
                print("\nSelect category number:")
                for i, cat in enumerate(INCOME_CATEGORIES, 1):
                    print(f"{i}. {cat}")

                cat_choice = int(input("Category: ")) - 1
                if 0 <= cat_choice < len(INCOME_CATEGORIES):
                    category = INCOME_CATEGORIES[cat_choice]
                    description = input("📝 Description (optional): ")
                    add_transaction('Income', amount, category, description)
                else:
                    print("❌ Invalid category selection")

            elif choice == '2':
                # Add Expense
                print("\n💸 Adding Expense Transaction")
                show_categories()
                amount = input("💵 Enter amount: $")
                print("\nSelect category number:")
                for i, cat in enumerate(EXPENSE_CATEGORIES, 1):
                    print(f"{i}. {cat}")

                cat_choice = int(input("Category: ")) - 1
                if 0 <= cat_choice < len(EXPENSE_CATEGORIES):
                    category = EXPENSE_CATEGORIES[cat_choice]
                    description = input("📝 Description (optional): ")
                    add_transaction('Expense', amount, category, description)
                else:
                    print("❌ Invalid category selection")

            elif choice == '3':
                # View Transactions
                try:
                    limit = int(input("📊 How many recent transactions to show? (default 10): ") or "10")
                    view_transactions(limit)
                except ValueError:
                    view_transactions()

            elif choice == '4':
                # Financial Summary
                get_financial_summary()

            elif choice == '5':
                # Monthly Summary
                get_monthly_summary()

            elif choice == '6':
                # Export to CSV
                filename = input("📁 Enter filename (default: moneywise_transactions.csv): ").strip()
                if not filename:
                    filename = "moneywise_transactions.csv"
                export_to_csv(filename)

            elif choice == '7':
                # Show Categories
                show_categories()

            elif choice == '8':
                # Transaction Statistics
                get_transaction_stats()

            elif choice == '9':
                # Exit
                print("👋 Thank you for using MoneyWise Solutions Finance Dashboard!")
                break

            else:
                print("❌ Invalid choice. Please select 1-9.")

        except KeyboardInterrupt:
            print("\n\n👋 Goodbye!")
            break
        except Exception as e:
            print(f"❌ An error occurred: {e}")

        input("\n⏸️  Press Enter to continue...")


In [ ]:
# STEP 9: SAMPLE DATA FOR TESTING
# ============================================================================

def add_sample_data():
    """Add sample transactions for testing"""
    print("📝 Adding sample data...")

    # Sample income transactions
    add_transaction('Income', 5000, 'Salary', 'Monthly salary')
    add_transaction('Income', 1200, 'Freelance', 'Web development project')
    add_transaction('Income', 500, 'Investment', 'Dividend payment')

    # Sample expense transactions
    add_transaction('Expense', 800, 'Housing', 'Monthly rent')
    add_transaction('Expense', 300, 'Food', 'Groceries and dining')
    add_transaction('Expense', 150, 'Transportation', 'Gas and car maintenance')
    add_transaction('Expense', 100, 'Utilities', 'Electricity bill')
    add_transaction('Expense', 80, 'Entertainment', 'Movies and subscriptions')

    print("✅ Sample data added successfully!")

# STEP 10: PROGRAM EXECUTION
# ============================================================================

if __name__ == "__main__":
    # Ask if user wants sample data
    add_sample = input("🔹 Add sample data for testing? (y/n): ").lower().strip()
    if add_sample == 'y':
        add_sample_data()

    # Start main program
    main()

🔹 Add sample data for testing? (y/n): y
📝 Adding sample data...
✅ Transaction added successfully! ID: b9175c02
✅ Transaction added successfully! ID: 4631027a
✅ Transaction added successfully! ID: b1e27c6d
✅ Transaction added successfully! ID: dd737a77
✅ Transaction added successfully! ID: 11674a78
✅ Transaction added successfully! ID: 3a38a562
✅ Transaction added successfully! ID: 41ca5dbb
✅ Transaction added successfully! ID: 89d64f58
✅ Sample data added successfully!
🏦 Welcome to MoneyWise Solutions Finance Dashboard!

🏦 MONEYWISE SOLUTIONS - FINANCE DASHBOARD
1️⃣  Add Income Transaction
2️⃣  Add Expense Transaction
3️⃣  View Recent Transactions
4️⃣  Generate Financial Summary
5️⃣  Generate Monthly Summary
6️⃣  Export to CSV
7️⃣  Show Categories
8️⃣  Transaction Statistics
9️⃣  Exit

🔹 Enter your choice (1-9): 1

💰 Adding Income Transaction

📋 AVAILABLE CATEGORIES
💰 INCOME CATEGORIES:
  1. Salary
  2. Freelance
  3. Investment
  4. Business
  5. Other Income

💸 EXPENSE CATEGORIES:
  